In [18]:
from platform import python_version


print(python_version())

3.7.4


In [19]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

#open a random image and check its size
im = Image.open("/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg")
#display.display(Image.open(str('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg')))
width, height = im.size 
print(width,height)

#resizing
im=im.resize((448,448),resample=0)
#im.show()


1024 684


To load the files as a tf.data.Dataset first create a dataset of the file paths:

In [20]:
#source: https://www.tensorflow.org/tutorials/load_data/images
def image_paths(dataset_path):
    """
    input: dpath of the whole image dataset
    returns: all of the imagepaths and all of the labels/classes
    as type tensorflow.python.framework.ops.EagerTensor
    """
    
    imagepaths=[]
    #os.walk returns a tuple of three elements: (root_dir_path, sub_dirs, files)
    
    #picking up the names of the subfiles equals the names of the classes 
    classes = os.walk(dataset_path).__next__()[1]
    
    # List each sub-directory (the classes)
    for c in classes:
        #seperate animal folder(=class)
        c_dir = os.path.join(dataset_path, c)
        #make a walk object for each folder
        walk = os.walk(c_dir).__next__()
        # Add each image (walk[2]) to the imagepaths list
        for sample in walk[2]:
            imagepath = os.path.join(c_dir,sample)
            imagepaths.append(imagepath)
            
            
    #cpnvert to tensfor  (No, not yet!)     
    #imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    
    
    return imagepaths     
        

In [21]:
imagepaths= image_paths('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages')
print(f'I have {len(imagepaths)} images in my dataset')

I have 37322 images in my dataset


Write a short pure-tensorflow function that converts a file paths to an (image_data, label) pair:

In [22]:
#source: https://www.tensorflow.org/tutorials/load_data/images
#get the label of one image

def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, '/')
  # The second to last is the class-directory
  return parts[-2] 

#get_label('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg')

In [23]:
#source: https://www.tensorflow.org/tutorials/load_data/images
#decode one image

[IMG_WIDTH, IMG_HEIGHT]=[448,448]
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [24]:
##source: https://www.tensorflow.org/tutorials/load_data/images
#return img,label for one image

def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

#process_path('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg')

In [25]:
# source: https://www.tensorflow.org/tutorials/load_data/images
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
list_ds = tf.data.Dataset.list_files(str('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/*/*'))
labeled_ds = list_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
type(labeled_ds)


tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [26]:
labeled_ds


<DatasetV1Adapter shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>

In [27]:
# source: https://www.tensorflow.org/tutorials/load_data/images
#randomly take one image and see the image and its label
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy())
  print("Label: ", label)

Image shape:  [[[0.56447834 0.54140407 0.54901963]
  [0.56966037 0.5532913  0.55926126]
  [0.5810399  0.56029415 0.5759804 ]
  ...
  [0.43744785 0.28842822 0.28450665]
  [0.45933136 0.32002816 0.3128678 ]
  [0.47382733 0.3365724  0.32872927]]

 [[0.56619793 0.5344065  0.5449851 ]
  [0.590005   0.56874156 0.57715863]
  [0.5702327  0.5494869  0.56517315]
  ...
  [0.46162835 0.31088918 0.30782738]
  [0.47517842 0.32853368 0.32382047]
  [0.4830603  0.33691618 0.3320361 ]]

 [[0.5885111  0.55389977 0.5654369 ]
  [0.5869787  0.5637274  0.57293606]
  [0.526626   0.5022074  0.51856804]
  ...
  [0.4711001  0.31650528 0.3107232 ]
  [0.48646763 0.33088046 0.32463482]
  [0.49739763 0.34140587 0.33516026]]

 ...

 [[0.19102599 0.21760584 0.23096812]
  [0.18901306 0.21437904 0.22814594]
  [0.21321689 0.23147596 0.24783662]
  ...
  [0.11891741 0.36789265 0.09993953]
  [0.12665927 0.366174   0.12231246]
  [0.14414571 0.3765649  0.15567482]]

 [[0.18592358 0.20372546 0.22031471]
  [0.16548762 0.1850954

In [28]:
#labeled_ds.batch(batch_size=10, drop_remainder=True)

In [29]:
#NO
def prepare_for_training(dataset,shuffle=False, BATCH_SIZE = 100 ):
    "this is a function that prepares the dataset for training"
    "input: tensor containing the labeled dataset"
    "output: dataset, shuffled and batched"
    
    if shuffle:
        #shuffle the whole dataset: 
        #for perfect shuffling, buffer should be >= size of ds
        ds = labeled_ds.shuffle(buffer_size=3732)
    
    #repeat forever
    #ds = ds.repeat()
    
    #batch it 
    #ds = ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    
    #prefetch lets the dataset fetch batched in the background
    #while the model is training 
    #ds = ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    
    
    return ds 

In [30]:
#NO
ds = prepare_for_training(labeled_ds)


UnboundLocalError: local variable 'ds' referenced before assignment

In [31]:
#SPLIT TRAIN,VAL,TEST
DATASET_SIZE=37322
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = labeled_ds
raw_train = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
raw_validation = test_dataset.skip(test_size)
raw_test = test_dataset.take(test_size)

In [32]:
print(raw_train)
print(raw_validation)
print(raw_test)

<DatasetV1Adapter shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>
<DatasetV1Adapter shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>
<DatasetV1Adapter shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>


**FORMAT DATA**

In [41]:
IMG_SIZE = 448
# All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label





In [42]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [43]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [44]:
for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

TensorShape([32, 448, 448, 3])

In [46]:
base_model = tf.keras.applications.vgg19.VGG19(include_top=False, 
                                               weights='imagenet', 
                                               input_tensor=None, 
                                               input_shape=None,
                                               pooling=None, 
                                               classes=1000)

In [47]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 14, 14, 512)


In [48]:
#FREEZING

base_model.trainable = False

